<a style="float:left;" href="https://colab.research.google.com/github/ClaudeCoulombe/VIARENA/blob/master/Labos/Lab-CIFAR_10-PMC/Identification_Objets-CIFAR_10-PMC.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
<br/>
### Rappel - Fonctionnement d'un carnet web iPython

* Pour exécuter le code contenu dans une cellule d'un carnet iPython, cliquez dans la cellule et faites (⇧↵, shift-enter);
* Le code d'un carnet iPython s'exécute séquentiellement de haut en bas de la page. Souvent, l'importation d'une bibliothèque Python ou l'initialisation d'une variable est préalable à l'exécution d'une cellule située plus bas. Il est donc recommandé d'exécuter les cellules en séquence. Enfin, méfiez-vous des retours en arrière qui peuvent réinitialiser certaines variables;
* Pour obtenir de l'information sur une fonction, utilisez la commande Python `help(`"nom de la fonction"`)`

# Identification d'objets à partir de photos - jeu de données CIFAR-10

# Perceptron multicouche - PMC

#### Inspiration: 

Michael Nielsen, http://neuralnetworksanddeeplearning.com/

Grant Sanderson, https://www.youtube.com/watch?v=Ilg3gGewQ5U

## Importation bibliothèques Python

In [ ]:
import os
import matplotlib.pyplot as plt

import tensorflow as tf
print("TensorFlow version:",tf.__version__)
import keras
print("Keras version:",keras.__version__)


## Fixer le hasard pour la reproductibilité

La mise au point de réseaux de neurones implique certains processus aléatoires. Afin de pouvoir reproduire et comparer vos résultats d'expérience, vous fixez temporairement l'état aléatoire grâce à un germe aléatoire unique.

Pendant la mise au point, vous fixez temporairement l'état aléatoire pour la reproductibilité mais vous répétez l'expérience avec différents germes ou états aléatoires et prenez la moyenne des résultats.
<br/>
##### **Note**: Pour un système en production, vous ravivez simplement l'état  purement aléatoire avec l'instruction `GERME_ALEATOIRE = None`

In [ ]:
import os

# Définir un germe aléatoire
GERME_ALEATOIRE = 21

# Définir un état aléatoire pour Python
os.environ['PYTHONHASHSEED'] = str(GERME_ALEATOIRE)

# Définir un état aléatoire pour Python random
import random
random.seed(GERME_ALEATOIRE)

# Définir un état aléatoire pour NumPy
import numpy as np
np.random.seed(GERME_ALEATOIRE)

# Définir un état aléatoire pour TensorFlow
import tensorflow as tf
tf.random.set_seed(GERME_ALEATOIRE)
os.environ['TF_DETERMINISTIC_OPS'] = '1'
os.environ['TF_CUDNN_DETERMINISTIC'] = '1'

print("Germe aléatoire fixé")

## Jeu de données - photos CIFAR-10
L'ensemble de données CIFAR-10 (Canadian Institute For Advanced Research) comporte 60 000 photographies en couleur de 32×32 pixels d'objets de 10 classes différentes. Il est relativement simple d'atteindre une précision de 80 %. On peut obtenir des performances de 90 % avec ces données avec des réseaux neuronaux convolutifs. 

* 0 : avion
* 1 : automobile
* 2 : oiseau
* 3 : chat
* 4 : cerfs
* 5 : chien
* 6 : grenouille
* 7 : cheval
* 8 : bateau
* 9 : camion


In [ ]:
# le jeu de données CIFAR-10
from keras.datasets import cifar10

dic_noms_etiquette = { 
    0 : "avion",
    1 : "automobile",
    2 : "oiseau",
    3 : "chat",
    4 : "cerf",
    5 : "chien",
    6 : "grenouille",
    7 : "cheval",
    8 : "bateau",
    9 : "camion",
}

# lire le jeu de données CIFAR-10 et le diviser entre
# les données d'entrainement et les données de test
(attributs_entrainement, classes_cibles_entrainement), (attributs_test, classes_cibles_test) = cifar10.load_data()

classes_cibles_entrainement = classes_cibles_entrainement.reshape((-1,))
classes_cibles_test = classes_cibles_test.reshape((-1,))

# Aperçu des tableaux de données
print()
print('Entraînement: attributs=%s, classes=%s' % (attributs_entrainement.shape, classes_cibles_entrainement.shape))
print('Test: attributs=%s, classes=%s' % (attributs_test.shape, classes_cibles_test.shape))

# Afficher les 24 premières images
print()
print("Quelques images avec leur étiquette...")
%matplotlib inline
# définir subplot
fig, axes = plt.subplots(nrows=4,ncols=6,figsize=(10,8))
for i_rangee in range(0,4):
    for i_colonne in range(0,6):
        axes[i_rangee,i_colonne].set_title(dic_noms_etiquette[int(classes_cibles_entrainement[i_rangee*6+i_colonne])],
                                           fontsize=10)
        axes[i_rangee,i_colonne].imshow(attributs_entrainement[i_rangee*6+i_colonne])
plt.show()


### Linéariser les attributs des images 32 x 32 pixels en 3072 attributs

In [ ]:
# Linéariser les attributs des images 32 x 32 pixels en 3072 attributs
nombre_attributs = attributs_entrainement.shape[1]*attributs_entrainement.shape[1]*3
print("nombre_attributs:",nombre_attributs)
attributs_entrainement = attributs_entrainement.reshape(attributs_entrainement.shape[0], nombre_attributs)
attributs_test = attributs_test.reshape(attributs_test.shape[0], nombre_attributs)
print("Linéarisation des attributs des images terminée!")

### Conversion des classes-cibles en vecteurs binaires à un bit discriminant

In [ ]:
# Conversion des classes-cibles en vecteurs binaires à un bit discriminant
from tensorflow.keras.utils import to_categorical
classes_cibles_entrainement = to_categorical(classes_cibles_entrainement)
classes_cibles_test = to_categorical(classes_cibles_test)
print("Conversion des classes-cibles en vecteurs binaires terminée!")

In [ ]:
classes_cibles_entrainement.shape

### Normalisation des images

In [ ]:
# Normalisation
def normalisation(entrainement, test):
    # convertir de nombres entiers à nombres décimaux
    entrainement_normalise = entrainement.astype('float32')
    test_normalise = test.astype('float32')
    # normalisation à un nombre entre 0 et 1
    entrainement_normalise = entrainement_normalise / 255.0
    test_normalise = test_normalise / 255.0
    return entrainement_normalise, test_normalise

attributs_entrainement, attributs_test = normalisation(attributs_entrainement, attributs_test)

print("Normalisation des images terminée!")

## Modèle de base

In [ ]:
# Construction du modèle

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

print("Création d'un modèle de base...")

dimension_entree = (nombre_attributs,)
nombre_classes_cibles = 10

modele_de_base = Sequential()

# Apprentissage et extraction des attributs
# Note: la première couche cachée comprime légèrement les données passant de 3072 attributs en entrée à 1024
modele_de_base.add(Dense(1024, activation='relu', input_shape=dimension_entree))
modele_de_base.add(Dense(512, activation='relu'))
modele_de_base.add(Dense(64, activation='relu'))
modele_de_base.add(Dense(64, activation='relu'))

# Classification des images
modele_de_base.add(Dense(nombre_classes_cibles, 
                         activation='softmax'))

print()
print("Description du modèle de base:")
modele_de_base.summary()


In [ ]:
# Compilation du modèle

print()
print("Compilation du modèle de base...")

modele_de_base.compile(loss="categorical_crossentropy", 
                       optimizer="adam", 
                       metrics=["accuracy"])


In [ ]:
attributs_entrainement.shape

In [ ]:
classes_cibles_entrainement.shape

## Évaluation du modèle avant entraînement


In [ ]:
# Évaluation du modèle avant entraînement

print()
print("Évaluation du modèle de base avant entraînement...")

resultats = modele_de_base.evaluate(attributs_test, classes_cibles_test, verbose=0)
print("Exactitude test: {:.2f}%".format(resultats[1]*100))
print("Erreur test: {:.2f}%".format(100-resultats[1]*100))

Voila qui est tout à fait normal. Puisque CIFAR-10 a 10 étiquettes de classe. Ainsi, en devinant au hasard, nous devrions obtenir une précision de 10%.

## Entraînement du modèle


In [ ]:
# Entraînement du modèle
import time

print()
print("Entraînement du modèle de base...")

moment_demarrage = time.process_time()

batch_size = 128
epochs = 15

traces_entrainement = modele_de_base.fit(attributs_entrainement,
                                         classes_cibles_entrainement,
                                         batch_size=batch_size,
                                         epochs=epochs,
                                         validation_split=0.1)

temps_execution = time.process_time() - moment_demarrage
print("Temps d'exécution:",round(temps_execution,2))


## Évaluation du modèle après entraînement


In [ ]:
# Évaluation du modèle
# Exactitude test: aux environs de 50%
print()
print("Évaluation du modèle...")

resultats = modele_de_base.evaluate(attributs_test, classes_cibles_test, verbose=0)
print("Exactitude test: {:.2f}%".format(resultats[1]*100))

## Affichage des courbes d'entraînement: erreur et exactitue


In [ ]:
## Affichage des courbes d'entraînement
hauteur = 8
plt.subplots(figsize=(hauteur,1.618*hauteur))
plt.subplot(211)
plt.title('Erreur entropie croisée')
plt.plot(traces_entrainement.history['loss'], color='blue', label='courbe entraînement')
plt.plot(traces_entrainement.history['val_loss'], color='orange', label='courbe test')
plt.ylabel("Erreur")
plt.xlabel("Nombre d'époques")
plt.legend()
plt.show()
# tracer l'exactitude
plt.subplots(figsize=(hauteur,1.618*hauteur))
plt.subplot(212)
plt.title('\nExactitude de la classification')
plt.plot(traces_entrainement.history['accuracy'], color='blue', label='courbe entraînement')
plt.plot(traces_entrainement.history['val_accuracy'], color='orange', label='courbe test')
plt.ylabel("Exactitude")
plt.xlabel("Nombre d'époques")
plt.legend()
plt.show()


In [ ]:
print("Carnet IPython exécution terminée!")
